In [39]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
engine = create_engine("sqlite:///Chinook.db")
db.run("SELECT * FROM artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [3]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [19]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [20]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [25]:
write_query({"question": "list of songs and artists are there"})

{'query': 'SELECT T1.Name AS SongName, T3.Name AS ArtistName FROM tracks AS T1 INNER JOIN albums AS T2 ON T1.AlbumId = T2.AlbumId INNER JOIN artists AS T3 ON T2.ArtistId = T3.ArtistId LIMIT 10'}

##### Execute Query

In [22]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [31]:
execute_query({"query": "SELECT T1.Name AS SongName, T3.Name AS ArtistName FROM tracks AS T1 INNER JOIN albums AS T2 ON T1.AlbumId = T2.AlbumId INNER JOIN artists AS T3 ON T2.ArtistId = T3.ArtistId LIMIT 10"})

{'result': '[(\'For Those About To Rock (We Salute You)\', \'AC/DC\'), (\'Put The Finger On You\', \'AC/DC\'), ("Let\'s Get It Up", \'AC/DC\'), (\'Inject The Venom\', \'AC/DC\'), (\'Snowballed\', \'AC/DC\'), (\'Evil Walks\', \'AC/DC\'), (\'C.O.D.\', \'AC/DC\'), (\'Breaking The Rules\', \'AC/DC\'), (\'Night Of The Long Knives\', \'AC/DC\'), (\'Spellbound\', \'AC/DC\')]'}

In [42]:
import pandas as pd
def print_df(state: State):
    sql_query = state["result"]
    df = pd.read_sql_query(sql_query, engine)
    return df

In [43]:
print_df({"result":"SELECT T1.Name AS SongName, T3.Name AS ArtistName FROM tracks AS T1 INNER JOIN albums AS T2 ON T1.AlbumId = T2.AlbumId INNER JOIN artists AS T3 ON T2.ArtistId = T3.ArtistId LIMIT 10"})

,SongName,ArtistName
0,For Those About To Rock (We Salute You),AC/DC
1,Put The Finger On You,AC/DC
2,Let's Get It Up,AC/DC
3,Inject The Venom,AC/DC
4,Snowballed,AC/DC
5,Evil Walks,AC/DC
6,C.O.D.,AC/DC
7,Breaking The Rules,AC/DC
8,Night Of The Long Knives,AC/DC
9,Spellbound,AC/DC
